# Паттерн стратегия через полноправные функции

## Описание паттерна

В книге «Паттерны проектирования» паттерн Стратегия описывается следующим образом:

        Определить семейство алгоритмов, инкапсулировать каждый из
        них и сделать их взаимозаменяемыми. Стратегия позволяет заменять
        алгоритм независимо от использующих его клиентов.

![image](strategy.png)

Как следует из приведенной UML-даграммы, для реализации патерна авторы предлагают использовать все блага ООП, в частности, наследование, абстрактные классы, полиморфизм и пр. Однако из-за наличия в Python полноправных функций, этот паттерн может быть реализован и без использования столь сложных концепций. В этом на и предстоит убедиться.

## Задача

Рассмотрим Интернет-магазин со следующими правилами формирования скидок:
- заказчику, имеющему не менее 1000 баллов лояльности, предоставляется глобальная скидка 5 % на весь заказ;
- на позиции, заказанные в количестве не менее 20 единиц в одном заказе, предоставляется скидка 10 %;
- на заказы, содержащие не менее 10 различных позиций, предоставляется глобальная скидка 7 %.
- к каждому заказу может быть применена только одна из данных скидок - скидка, дающая наибольший бонус.

Также отдел аналитики нашего Интернет-магазина хочет собирать статистику по популярности использования каждой стратегии начисления скидки.

Наша задача, используя структы ниже, реализовать механизм расчета оптимальной скидки для каждого заказа и сбора статистика для отдела аналитики.

**Структуры для выполнения**:

In [3]:
from typing import Optional, Callable
from dataclasses import dataclass

In [4]:
@dataclass
class Item:
    label: str
    price: float
    amount: int

In [6]:
@dataclass
class Customer:
    customer_id: int
    username: str
    loyalty_points: int

In [5]:
class Order:
    customer: Customer
    order: list[Item]
    price: float
    promotion: Optional[Callable] = None

    def __init__(self, 
        customer: Customer,
        order: list[Item],
        promotion: Optional[Callable] = None
    ) -> None:
        self.customer = customer
        self.order = list(order)
        self.promotion = promotion
        self.price = self._compute_price()

    def _compute_price(self) -> float:
        price = sum(item.price * item.amount for item in self.order)

        if self.promotion:
            price -= self.promotion(self.order)

        return price